<a href="https://colab.research.google.com/github/cunhamaicon/petr4/blob/master/petr4_rnn3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importação dos pacotes ncessários:

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import model_from_json
from tensorflow.keras import regularizers
import math
from sklearn.metrics import mean_squared_error
import os

In [22]:
#FUNÇÕES

#Filtro
def binario(valor):
    bi=[]
    for c in range(1,len(valor)):
        if valor[c]>valor[c-1]:
            bi.append(1)
        elif valor[c]<valor[c-1]:
            bi.append(0)
        elif valor[c]==valor[c-1]:
            bi.append(0.5)
    return bi

def filtro(real,predito):   
    d=[]
    g=0
    for c in range(0,len(predito)):
        if predito[c]==-1 or real[c]==-1:
            d.append(c-g)
            g+=1
    for c in range(0,len(d)):
        del(real[d[c]])
        del(predito[d[c]])
            
    return real,predito

#FORMATAÇÃO DOS DADOS
def fit_dados(dados,dados_ptreino):
    X_train = []
    y_train = []
    for i in range(tick_treino,dados_ptreino):
        X_train.append(dados[i-tick_treino:i])
        y_train.append(dados[i,coluna_fechamento])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (dados_ptreino-tick_treino,tick_treino,n_entradas))
    
    return X_train,y_train

###Definição de parâmetros:

In [23]:
new_model=1
#-----//////---------/////---------
nome_model="EP=85_Train=700_Func=tahn_train_gradiente=300_tickTrei=16_time=15 min.json_fDropout_line.json"
nome_peso="EP=85_Train=700_Func=tahn_train_gradiente=300_tickTrei=16_time=15 min.json_fDropout_line.h5"
#-----//////---------/////---------
epocas=50
func="linear"
train_gradiente=16
dados_ptreino=2000
dados_ptest=500
tick_treino=90
div_dataset=70

entradas=[0,1,2,3]
n_entradas=len(entradas)
coluna_fechamento=3


###Criando os bancos:

In [24]:
detalhamento="n_epocas= {}; dados_treinados={}; train_gradiente={}; time=5 min; func={};sem_Dropout".format(epocas,dados_ptreino,train_gradiente,tick_treino,func)

dataset = pd.read_csv(os.path.join('GBPCHF',r"C:\Users\edno2\Desktop\Rede_Binária\GBPCHF_2020_05.csv"))

### Treinamento e Teste

Separando dois bancos, um de treinamento e um de teste:

In [25]:
pocent_treino=(len(dataset)//100)*div_dataset
pocent_test=len(dataset)-pocent_treino


dataset_train = dataset.iloc[:pocent_treino,1:]
training_set = dataset_train.iloc[len(dataset_train)-dados_ptreino:].values.astype(float)

dataset_test = dataset.iloc[pocent_treino:,1:]
test_set = dataset_test.iloc[0:dados_ptest].values.astype(float)

dataset_train.head(-1)

,ABERTURA,MAXIMA,MINIMA,FECHAMENTO
0,1.20964,1.20982,1.20959,1.20961
1,1.20961,1.20971,1.20956,1.20961
2,1.20961,1.21007,1.20958,1.20982
3,1.20980,1.20992,1.20980,1.20988
4,1.20987,1.21003,1.20977,1.21001
...,...,...,...,...
3004,1.20698,1.20698,1.20683,1.20691
3005,1.20692,1.20707,1.20692,1.20707
3006,1.20708,1.20721,1.20687,1.20695
3007,1.20696,1.20697,1.20675,1.20675


In [26]:
#Fazendo a transformação dos dados para ficar no intervalo [0,1]

sc = MinMaxScaler(feature_range = (0, 1))

training_set_scaled = sc.fit_transform(training_set)
real_stock_price = sc.fit_transform(test_set)

Cada ciclo de treinamento deve ter o tamanho "size" definido anteriormente representando as "size" entradas anterioes. O conjunto de treinamento "X_train" deve emparelhar todas essas entradas. "y_train" é a saída da rede, o próximo elemento depois das "size" entradas:

In [27]:
#FORMATANDO OS DADOS 

X_train,y_train=fit_dados(training_set_scaled,dados_ptreino)

X_test,real_preco=fit_dados(real_stock_price,dados_ptest)

## Construindo a RNN

In [30]:
def regressor():
  regressor = Sequential()

  regressor.add(LSTM(units = 33,activation=func,  return_sequences = True, kernel_regularizer = regularizers.l2(), input_shape = (tick_treino, n_entradas)))
  #regressor.add(Dropout(0.2))

  
  regressor.add(LSTM(units = 33,return_sequences = True, kernel_regularizer = regularizers.l2 (0.0001)))
 
  
  regressor.add(LSTM(units = 33, return_sequences = True))
 

  regressor.add(LSTM(units = 33))
 

  regressor.add(Dense(1, activation='sigmoid'))

 
  regressor.compile(optimizer = "adam", loss = 'mean_squared_error')
  
  return regressor

Mostrando as informações da rede:

In [31]:
regressor = regressor()
print(regressor.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 90, 33)            5016      
_________________________________________________________________
lstm_1 (LSTM)                (None, 90, 33)            8844      
_________________________________________________________________
lstm_2 (LSTM)                (None, 90, 33)            8844      
_________________________________________________________________
lstm_3 (LSTM)                (None, 33)                8844      
_________________________________________________________________
dense (Dense)                (None, 1)                 34        
Total params: 31,582
Trainable params: 31,582
Non-trainable params: 0
_________________________________________________________________
None


### Treinando a rede

In [ ]:
history = regressor.fit(X_train, y_train, epochs = epocas, batch_size =train_gradiente)

In [14]:
# serialize model to JSON
regressor_json = regressor.to_json()
with open("ABERTURA_BINARIO_INDICADORES(MACD,STOC,BI)_100EPOCAS_LINEAR_24TICKS.json", "w") as json_file:
    json_file.write(regressor_json)
# serialize weights to HDF5
regressor.save_weights("ABERTURA_BINARIO_INDICADORES(MACD,STOC,BI)_100EPOCAS_LINEAR_24TICKS.h5")
print("Saved model to disk")

Saved model to disk


In [50]:
if new_model==1:
    predicted_stock_price = regressor.predict(X_test) 
     

if new_model==0:
    # load json and create model
    json_file = open(nome_model, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(nome_peso)
    print("Loaded model from disk")
    
    predicted_stock_price = loaded_model.predict(X_test)

## Visualização dos resultados

In [51]:
if new_model==1:
    predicted_stock_price = regressor.predict(X_test) 

if new_model==0:
    # load json and create model
    json_file = open(nome_model, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(nome_peso)
    print("Loaded model from disk")
    
    predicted_stock_price = loaded_model.predict(X_test)

In [52]:
#TRANSFORMANDO EM BINARIO

x2=binario(predicted_stock_price)
x1=binario(real_preco)
real,predito=filtro(x1,x2)  

In [53]:
#tp e tn são os corretos
tn, fp, fn, tp=confusion_matrix(real,predito).ravel()
acuracia=((tp+tn)/(tp+tn+fp+fn))*100
print("n-----------------------Verdadeiro Positivo:{}\nVerdadeiro Negativo:{}\n-----------------------\nFalso Positivo:{}\nFalso Negativo:{}\n-----------------------".format(tp,tn,fp,fn))
print("Acurracia:{:.2f}".format(acuracia))

ValueError: Classification metrics can't handle a mix of continuous and binary targets